In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
from oceanoobsbrasil.bd import GetData
from oceanoobsbrasil.utils import *
from oceanoobsbrasil.get_pe_buoy import PEBuoy

import os
from dotenv import load_dotenv

from bs4 import BeautifulSoup
import re
import chromedriver_binary
%load_ext autoreload
%autoreload 2

In [4]:
options = Options()
options.add_argument('-headless')
driver = webdriver.Chrome(options=options)
url = "https://www.windguru.cz/station/481"

In [6]:
driver.get(url)

In [7]:
soup=BeautifulSoup(driver.page_source, 'html.parser')

In [10]:
l=soup.find(attrs={'class': 'wgs_temp_value'}).text
l

'32.1'

In [11]:
l=soup.find(attrs={'class': 'wgs_wind_avg_value'}).text
l

'2.5'

In [39]:
l=soup.find(attrs={'class': 'wgs_wind_dir_numvalue'}).text
l

'41°'

In [41]:
l=soup.find(attrs={'class': 'wgs_last_time'}).text
date_time = datetime.strptime(l[:15]+"0", "%Y-%m-%d %H:%M")

'2021-09-13 19:26:01 -03'

In [46]:
datetime.strptime(l[:15]+"0", "%Y-%m-%d %H:%M")

datetime.datetime(2021, 9, 13, 19, 20)

In [33]:
station_id = np.array([911, 912, 913, 914, 915, 916, 917, 918])
name = np.array(["Rio de Janeiro | Mangaratiba", "Praça do O | BRKITE",
    "Barra da Tijuca, K7", "Ipanema | Kitesurf Rio", "Ponta de Campina, Bodete KiteClub",
    "Cumbuco Ocean View Aparthotel", "Vila Coqueiros Boutique Hotel",
    "Taiba Ceara"])

url = np.array([607, 520, 530, 481, 2556, 2653, 634, 2001])

lat = np.array([-22.9826,-23.0143, -23.0168,-22.9879, -7.0521, -22.9826, -22.9826, -3.4784])

lon = np.array([-44.0364, -43.3169, -43.307, -43.2061, -34.8246, -44.0364, -44.0364, -38.926])

data = np.array([station_id, name, url, lat, lon])


In [34]:
df = pd.DataFrame(data).T

In [35]:
df.columns = ['id', 'name', 'url', 'lat', 'lon']

In [36]:
df['data_type'] = 'meteorological_station'
df['institution'] = 'windguru'
df

,id,name,url,lat,lon,data_type,institution
0,911,Rio de Janeiro | Mangaratiba,607,-22.9826,-44.0364,meteorological_station,windguru
1,912,Praça do O | BRKITE,520,-23.0143,-43.3169,meteorological_station,windguru
2,913,"Barra da Tijuca, K7",530,-23.0168,-43.307,meteorological_station,windguru
3,914,Ipanema | Kitesurf Rio,481,-22.9879,-43.2061,meteorological_station,windguru
4,915,"Ponta de Campina, Bodete KiteClub",2556,-7.0521,-34.8246,meteorological_station,windguru
5,916,Cumbuco Ocean View Aparthotel,2653,-22.9826,-44.0364,meteorological_station,windguru
6,917,Vila Coqueiros Boutique Hotel,634,-22.9826,-44.0364,meteorological_station,windguru
7,918,Taiba Ceara,2001,-3.4784,-38.926,meteorological_station,windguru


In [37]:
db = GetData()

In [38]:
db.post(table='stations', df=df)

In [47]:
from oceanoobsbrasil.get_wind_guru import WindGuru

In [94]:
w = WindGuru()

In [95]:
w.get()

Rio de Janeiro | Mangaratiba
[<selenium.webdriver.remote.webelement.WebElement (session="a634f511e9db0305fe95c419931b761b", element="f377a343-c613-4dd1-87e4-d594e598d239")>]


/home/soutobias/code/soutobias/personal_codes/remobs/oceanoobsbrasil/oceanoobsbrasil/get_wind_guru.py:71: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  columns = ['date_time', 'wspd', 'wdir', 'sst']


          date_time                                               wspd wdir  \
0  2021-09-13 23:20  [<selenium.webdriver.remote.webelement.WebElem...        

  sst station_id  
0            911  


ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'WebElement'
[SQL: INSERT INTO data_stations (date_time, wspd, wdir, sst, station_id) VALUES (%(date_time)s, %(wspd)s, %(wdir)s, %(sst)s, %(station_id)s)]
[parameters: {'date_time': '2021-09-13 23:20', 'wspd': [<selenium.webdriver.remote.webelement.WebElement (session="a634f511e9db0305fe95c419931b761b", element="f377a343-c613-4dd1-87e4-d594e598d239")>], 'wdir': '', 'sst': '', 'station_id': '911'}]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [103]:
soup=BeautifulSoup(w.driver.page_source, 'html.parser')

In [102]:
w.result.wspd.iloc[0][0].text

''

In [90]:
w.soup.find(attrs={'class': 'wgs_temp_value'})

<span class="wgs_temp_value"></span>

In [69]:
w.soup.find(attrs={'class': 'wgs_last_time'})

<div class="wgs_last_time"></div>

In [81]:
datetime.strftime(datetime.utcnow(), '%Y-%m-%d %H:%M')[:-1]+"0"

'2021-09-13 23:10'